# Quora Insincere Questions Logistic Regression with Upsampling

In [1]:
# import packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
import nltk
import re

from gensim import corpora, models, similarities

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

np.random.seed(27)

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


#### Text Pre-processing

In [3]:
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(contractions.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return contractions[match.group(0)]
    return c_re.sub(replace, text)

In [4]:
# function to clean and lemmatize text and remove stopwords
from gensim.parsing.preprocessing import preprocess_string

def gensim_preprocess(docs):
    docs = [expandContractions(doc) for doc in docs]
    docs = [preprocess_string(text) for text in docs]
    return pd.Series(docs)

gensim_preprocess(train.question_text.iloc[10:15])

0                                              [femin]
1                              [calgari, flame, found]
2       [dumbest, possibl, true, explan, trump, elect]
3    [us, extern, hard, disk, data, storag, data, a...
4    [live, home, boyfriend, love, boyfriend, home,...
dtype: object

In [5]:
# apply text-preprocessing function to training set
%time train_corpus = gensim_preprocess(train.question_text)

CPU times: user 3min 28s, sys: 3.22 s, total: 3min 32s
Wall time: 3min 34s


In [6]:
# create ngrams
ngram_phraser = models.Phrases(train_corpus, threshold=1)
ngram = models.phrases.Phraser(ngram_phraser)
#print example
print(ngram[train_corpus[0]])

# apply model to corpus
train_texts = [ngram[token] for token in train_corpus]

['quebec', 'nationalist', 'provinc', 'nation']


In [7]:
# preparing ngrams for modeling
train_texts = [' '.join(text) for text in train_texts]
train['ngrams'] = train_texts

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(use_idf=True,
                     min_df=50,
                     max_features=20000,
                     ngram_range=(1,2),
                     norm='l1',
                     smooth_idf=True).fit(train_texts)

In [9]:
tv_matrix = tv.transform(train_texts)

X_train, X_test, y_train, y_test = train_test_split(tv_matrix, train.target, test_size=0.2)

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_train, y_train = ros.fit_sample(X_train, y_train)

### Logistic Regression Baseline Model

In [10]:
#X_test = vectorizer.transform(X_test)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

print('Logistic Regression Score: ', lr.score(X_test, y_test))

Logistic Regression Score:  0.8492257632309312


In [11]:
y_ = lr.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_))

              precision    recall  f1-score   support

           0       0.99      0.85      0.91    245069
           1       0.27      0.82      0.40     16156

   micro avg       0.85      0.85      0.85    261225
   macro avg       0.63      0.83      0.66    261225
weighted avg       0.94      0.85      0.88    261225



In [15]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, y_))

,0,1
0,208617,36452
1,2934,13222


In [13]:
# preprocessing/lemmatizing/stemming test data
%time test_corpus = gensim_preprocess(test.question_text)
test_texts = [ngram[token] for token in test_corpus]

test_texts = [' '.join(text) for text in test_texts]
test['ngrams'] = test_texts
test.head()

CPU times: user 6.08 s, sys: 22.3 ms, total: 6.11 s
Wall time: 6.11 s


,qid,question_text,ngrams
0,00014894849d00ba98a9,My voice range is A2-C5. My chest voice goes u...,voic rang chest_voic goe includ sampl higher c...
1,000156468431f09b3cae,How much does a tutor earn in Bangalore?,tutor earn bangalor
2,000227734433360e1aae,What are the best made pocket knives under $20...,best pocket_knive
3,0005e06fbe3045bd2a92,Why would they add a hypothetical scenario tha...,add hypothet_scenario that’ imposs happen link...
4,00068a0f7f41f50fc399,What is the dresscode for Techmahindra freshers?,dresscod techmahindra fresher


In [14]:
# # logistic regression on test data
# prediction = lr.predict(tv.transform(test.ngrams))

# submission = pd.DataFrame({'qid':test.qid, 'prediction':prediction})
# submission.to_csv('submission.csv', index=False)
# submission.head()

By using upsampling, we were able to improve the submission's F1 score to 0.524 from the previous best score of 0.483 from logistic regression with downsampling.